In [1]:
from numpy.random import seed
seed(3363)
from tensorflow import set_random_seed
set_random_seed(3363)

In [2]:
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, InputLayer, LeakyReLU, BatchNormalization, Dropout, GlobalAveragePooling2D
from keras.applications.densenet import DenseNet121
from sklearn.metrics import roc_auc_score
from dataset_batch import load_train_data, load_test_data
import tensorflow as tf

Using TensorFlow backend.


In [3]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

class DenseNetModel():

    def __init__(self, input_dim=(224,224,3), output_dim=14, learning_rate=0.00001, epochs=5, drop_out=0.3,
                 freeze=True):

        # parms:
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.drop_out = drop_out
        self.trainable = not freeze

        # Define DenseNet
        self.model = Sequential()

        base_model = DenseNet121(weights='imagenet', include_top=False, pooling='avg', input_shape=self.input_dim)

        # Freeze the model first
        base_model.trainable = self.trainable

        self.model.add(base_model)

        self.model.add(Dropout(self.drop_out))
        self.model.add(Dense(512))
        self.model.add(Dropout(self.drop_out))
        self.model.add(Dense(self.output_dim, activation = 'sigmoid'))

        auc_roc = as_keras_metric(tf.metrics.auc)
        recall = as_keras_metric(tf.metrics.recall)
        precision = as_keras_metric(tf.metrics.precision)

        self.model.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                           metrics = ['binary_accuracy', 'mae', auc_roc, recall, precision])
        self.model.summary()

        # Image augmentation
        self.core_idg = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip=True, 
                              vertical_flip=False, 
                              height_shift_range=0.05, 
                              width_shift_range=0.15, 
                              rotation_range=10, 
                              shear_range=0.2,
                              fill_mode='constant',
                              cval=1,
                              zoom_range=[0.85,1.20])

    # Data standardization
    def standardize(self, x):
        self.core_idg.fit(x)
        return

    # fit the data
    def fit(self, x, y, vx, vy):
        print ("Start Training model")
        X_train, y_train, X_test, y_test = x, y, vx, vy
        hist = self.model.fit_generator(
            (self.core_idg.flow(X_train, y_train, batch_size = 25)),
            validation_data = self.core_idg.flow(X_test, y_test), epochs=self.epochs)
        print ("Done Training model")
        return hist

    # data did preprocessing            
    def inference(self, x):
        return self.model.predict(x)

    # make predicition
    def score(self, x, y):
        y_pred = self.predict(x)
        return roc_auc_score(y, y_pred, average = "macro")

    # pain data without preprocessing
    def predict(self, x):
        x = self.core_idg.standardize(x)
        return self.inference(x)

    def save_weight(self, path = 'baseline.h5'):
        print ("Start Saving model")
        self.model.save(path)
        print ("Done Saving model")
        return

    def load_weight(self, path = 'baseline.h5'):
        print ("Start Loading model")
        self.model.load_weights(path)
        print ("Done Loading model")
        return

In [4]:
if __name__ == "__main__":

    model = DenseNetModel(input_dim=(224,224,3), epochs=5, drop_out=0.4, freeze=False)

    X_vali, y_vali = load_train_data(min_cnt=0, max_cnt=800)
    for itera in range(40):
        for base in range(800, 10001, 2400):
            X_train, y_train = load_train_data(min_cnt=base, max_cnt=base+2399)
            model.standardize(X_train)
            model.fit(X_train, y_train, X_vali, y_vali)
        print("Iter",itera,"AVG AUC:",model.score(X_vali, y_vali))

    # Saving model
    model.save_weight()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 1024)              7037504   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 14)                7182      
Total params: 7,569,486
Trainable params: 7,485,838
Non-trainable params: 83,648
_________________________________________________________________
Loading Training Images and Labels
0.0 % done
5.0 % done
10.0 % done


/home/owner/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 0 to 800
(801, 224, 224, 3) (801, 14)
Loading Training Images and Labels
0.0 % done
5.0 % done
10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 800 to 3199
(2400, 224, 224, 3) (2400, 14)
Start Training model
Epoch 1/5
96/96 [==============================] - 79s 821ms/step - loss: 0.2461 - binary_accuracy: 0.9381 - mean_absolute_error: 0.0923 - auc: 0.6357 - recall: 0.9896 - precision: 0.0471 - val_loss: 0.4477 - val_binary_accuracy: 0.8850 - val_mean_absolute_error: 0.1383 - val_auc: 0.6586 - val_recall: 0.

Epoch 5/5
81/81 [==============================] - 42s 520ms/step - loss: 0.2193 - binary_accuracy: 0.9463 - mean_absolute_error: 0.0895 - auc: 0.6458 - recall: 0.8695 - precision: 0.0491 - val_loss: 0.2480 - val_binary_accuracy: 0.9475 - val_mean_absolute_error: 0.0820 - val_auc: 0.6468 - val_recall: 0.8721 - val_precision: 0.0491
Done Training model
Iter 0 AVG AUC: 0.5139291615585705
Loading Training Images and Labels
0.0 % done
5.0 % done


/home/owner/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


...

10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 800 to 3199
(2400, 224, 224, 3) (2400, 14)
Start Training model
Epoch 1/5
96/96 [==============================] - 49s 511ms/step - loss: 0.0973 - binary_accuracy: 0.9641 - mean_absolute_error: 0.0499 - auc: 0.8340 - recall: 0.9948 - precision: 0.0494 - val_loss: 0.3158 - val_binary_accuracy: 0.9402 - val_mean_absolute_error: 0.0690 - val_auc: 0.8340 - val_recall: 0.9948 - val_precision: 0.0494
Epoch 2/5
96/96 [==============================] - 49s 509ms/step - loss: 0.0718 - binary_accuracy: 0.9719 - mean_absolute_error: 0.0413 - auc: 0.8341 - recall: 0.9948 - precision: 0.0494 - val_loss: 0.3472 - val_binary_accuracy: 0.9287 - val_mean_absolute_error: 0.0820 - val_auc: 0.8341 - val_recall: 0.9948 - val_precision: 0

Done Training model
Iter 38 AVG AUC: 0.6637979171618353
Loading Training Images and Labels
0.0 % done
5.0 % done


/home/owner/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


10.0 % done
15.0 % done
20.0 % done
25.0 % done
30.0 % done
35.0 % done
40.0 % done
45.0 % done
50.0 % done
55.0 % done
60.0 % done
65.0 % done
70.0 % done
75.0 % done
80.0 % done
85.0 % done
90.0 % done
95.0 % done
100.0 % done
Done Loading Training Images and Labels from 800 to 3199
(2400, 224, 224, 3) (2400, 14)
Start Training model
Epoch 1/5
96/96 [==============================] - 49s 512ms/step - loss: 0.0948 - binary_accuracy: 0.9646 - mean_absolute_error: 0.0484 - auc: 0.8359 - recall: 0.9949 - precision: 0.0494 - val_loss: 0.4018 - val_binary_accuracy: 0.9299 - val_mean_absolute_error: 0.0792 - val_auc: 0.8359 - val_recall: 0.9950 - val_precision: 0.0494
Epoch 2/5
96/96 [==============================] - 49s 509ms/step - loss: 0.0677 - binary_accuracy: 0.9738 - mean_absolute_error: 0.0396 - auc: 0.8359 - recall: 0.9950 - precision: 0.0494 - val_loss: 0.3196 - val_binary_accuracy: 0.9349 - val_mean_absolute_error: 0.0765 - val_auc: 0.8360 - val_recall: 0.9950 - val_precision: 0

Done Training model
Iter 39 AVG AUC: 0.6178113320757653
Start Saving model
Done Saving model
